# Импорты библиотек

In [1]:
import sympy
import plotly.express as px
import numpy

# Входные данные

In [2]:
x = sympy.symbols('x')
f = (x - 1) * (x - 0.0001)

f_numeric = sympy.lambdify(x, f, 'numpy')
f1_numeric = sympy.lambdify(x, sympy.diff(f, x), 'numpy')
f2_numeric = sympy.lambdify(x, sympy.diff(f, x, 2), 'numpy')

a = -1.5
b = 1.5
epsilon = 1e-6

# График функции

In [3]:
xs = numpy.linspace(a, b, 1000)
ys = f_numeric(xs)
px.line(x=xs, y=ys).show()

# Решение уравнения $f(x) = 0$

## 1. Отделение корней

In [4]:
def isolate_roots(f_numeric: callable, a: float, b: float, steps: int = 10) -> list[tuple[float, float]]:
    xs = numpy.linspace(a, b, steps + 1)
    ys = f_numeric(xs)

    intervals = []
    for i in range(steps):
        if numpy.isnan(ys[i]) or numpy.isnan(ys[i+1]):
            continue
        if ys[i] == 0:
            intervals.append((xs[i], xs[i]))
        elif ys[i] * ys[i+1] < 0:
            intervals.append((xs[i], xs[i+1]))

    return intervals

In [5]:
intervals = isolate_roots(f_numeric, a, b, 100)
intervals

[(np.float64(0.0), np.float64(0.030000000000000027)),
 (np.float64(0.9899999999999998), np.float64(1.02))]

## 2. Нахождение отдельного корня

Метод касательных:</br>
$\displaystyle \={x}_n = \={x}_{n - 1} - \frac{f(\={x}_{n - 1})}{f'(\={x}_{n - 1})}, n = 1, 2, \ldots \\$
Метод хорд:</br>
$\displaystyle x_n = x_{n - 1} - \frac{f(x_{n - 1}) (\={x}_{n - 1} - x_{n - 1})}{f(\={x}_{n - 1}) - f(x_{n - 1})}, n = 1, 2, \ldots$

In [6]:
def tangent_chord_method(f: callable, f1: callable, f2: callable, a: float, b: float, epsilon : float = 1e-8, max_iter: int = 100) -> float:
    fa, fb = f(a), f(b)
    f2a, f2b = f2(a), f2(b)
    xbar = 0

    if fa * f2a > 0:
        xbar = a
        x = b
    elif fb * f2b > 0:
        xbar = b
        x = a

    for _ in range(max_iter):
        f_xbar = f(xbar)
        f1_xbar = f1(xbar)

        if abs(f1_xbar) < 1e-14:
            break

        xbar_next = xbar - f_xbar / f1_xbar

        fx = f(x)
        denom = (f_xbar - fx)
        if abs(denom) < 1e-14:
            x_next = (x + xbar) / 2
        else:
            x_next = x - fx * (xbar - x) / denom

        if abs(x_next - xbar_next) < epsilon:
            return (x_next + xbar_next) / 2

        xbar, x = xbar_next, x_next

    return (xbar + x) / 2

In [7]:
roots = []
for alpha, beta in intervals:
    if alpha == beta:
        alpha -= 1e-6
        beta += 1e-6
    root = tangent_chord_method(f_numeric, f1_numeric, f2_numeric, alpha, beta, epsilon==1e-8)
    roots.append(root)

roots

[np.float64(0.0001), np.float64(1.0)]

## 3. Проверка

In [8]:
for root in roots:
    print(root, f.subs(x, root))

0.0001 0
1.0 0
